>### 🚩 *Create a free WhyLabs account to get more value out of whylogs!*<br>
>*Did you know you can store, visualize, and monitor whylogs profiles with the [WhyLabs Observability Platform](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Pyspark_Profiling)? Sign up for a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Pyspark_Profiling) to leverage the power of whylogs and WhyLabs together!*

# PySpark Integration
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/integrations/Pyspark_Profiling.ipynb)


Hi! Perhaps you're already feeling confident with our library, but you really wish there was an easy way to plug our profiling into your existing PySpark jobs. Well, glad you've made it here, because this is what we are going to cover in this example notebook 😃

If you wish to have other insights on how to use whylogs, feel free to check our [other existing examples](https://github.com/whylabs/whylogs/tree/mainline/python/examples), as they might be extremely useful!

## Installing the extra dependency

As we want to enable users to have exactly what they need to use from whylogs, the `pyspark` integration comes as an extra dependency. In order to have it available, simply uncomment and run the following cell:

In [ ]:
# Note: you may need to restart the kernel to use updated packages.
%pip install 'whylogs[spark]'

## Initializing a SparkSession

Here we will initialize a SparkSession. I'm also setting the `pyarrow` execution config, because it makes our methods even more performant.

>**IMPORTANT**: Make sure you have Spark 3.0+ available in your environment, as our implementation relies on it for a smoother integration

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('whylogs-testing').getOrCreate()
arrow_config_key = "spark.sql.execution.arrow.pyspark.enabled"
spark.conf.set(arrow_config_key, "true")

## Reading the data

For the sake of simplicity (and computational efforts, so you can run this notebook from your local machine), we will read the Wine Quality dataset, available in this URL: "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv".

In [4]:
from pyspark import SparkFiles

data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
spark.sparkContext.addFile(data_url)

In [5]:
spark_dataframe = spark.read.option("delimiter", ";").option("inferSchema", "true").csv(SparkFiles.get("winequality-red.csv"), header=True)

In [6]:
spark_dataframe.show(n=1, vertical=True)

-RECORD 0----------------------
 fixed acidity        | 7.4    
 volatile acidity     | 0.7    
 citric acid          | 0.0    
 residual sugar       | 1.9    
 chlorides            | 0.076  
 free sulfur dioxide  | 11.0   
 total sulfur dioxide | 34.0   
 density              | 0.9978 
 pH                   | 3.51   
 sulphates            | 0.56   
 alcohol              | 9.4    
 quality              | 5      
only showing top 1 row



In [7]:
spark_dataframe.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)



## Profiling the data with whylogs

Now that we have a Spark DataFrame in place, let's see how easy it is to profile our data with whylogs.

In [8]:
from whylogs.api.pyspark.experimental import collect_column_profile_views

column_views_dict = collect_column_profile_views(spark_dataframe)

Yeap. It's done. It is **that** easy.

But what do we get with a `column_views_dict`?

In [9]:
print(column_views_dict)

{'alcohol': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc4128910>, 'chlorides': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc40644f0>, 'citric acid': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc423ab90>, 'density': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc41c64d0>, 'fixed acidity': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc41c5030>, 'free sulfur dioxide': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc419b370>, 'pH': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc5370550>, 'quality': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc53719f0>, 'residual sugar': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc53eb4c0>, 'sulphates': <whylogs.core.view.column_profile_view.ColumnProfileView object at 0x7a9dc53e9f00>, 'total sulfur dioxide

It is a dictionary with one `ColumnProfileView` object per column in your dataset. And we can inspect some of the metrics on each one of them, such as the counts for a given column

In [10]:
column_views_dict["density"].get_metric("counts").n.value, spark_dataframe.count()

(1599, 1599)

Or their `mean` value:

In [11]:
column_views_dict["density"].get_metric("distribution").mean.value

0.9967466791744841

And now let's check how accurate whylogs did store that `mean` calculation.

In [12]:
from pyspark.sql.functions import mean
spark_dataframe.select(mean("density")).show()

+------------------+
|      avg(density)|
+------------------+
|0.9967466791744831|
+------------------+



It is not the literal exact value, but it gets really close, right? That is because we are not extracting the exact information, but we are also **not sampling** the data. `whylogs` will look at **every data point** and *statistically* decide wether or not that data point is relevant to the final calculation.

Is it just me or this is extremely powerful? Yes, it is.

> "Cool! But what can I do with a bunch of `ColumnProfileView`'s from my Dataset? I want to see everything together

Well, you've come to the right place, because we will inspect the next method that does just that!

In [13]:
from whylogs.api.pyspark.experimental import collect_dataset_profile_view

dataset_profile_view = collect_dataset_profile_view(input_df=spark_dataframe)

Yes, that easy. You now have a `DatasetProfileView`. As you might have seen from other example notebooks in our repo, you can turn this *lightweight* object into a pandas DataFrame, and visualize all the important metrics that we've profiled, like this:

In [14]:
import pandas as pd

dataset_profile_view.to_pandas().head()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,counts/true,distribution/max,distribution/mean,...,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor,frequent_items/frequent_strings,ints/max,ints/min
column,,,,,,,,,,,,,,,,,,,,,
alcohol,65.000010,65.000000,65.003256,0,1599,0,0,0,14.90000,10.422983,...,SummaryType.COLUMN,0,1599,0,0,0,0,NaN,NaN,NaN
chlorides,153.000058,153.000000,153.007697,0,1599,0,0,0,0.61100,0.087467,...,SummaryType.COLUMN,0,1599,0,0,0,0,NaN,NaN,NaN
citric acid,80.000016,80.000000,80.004010,0,1599,0,0,0,1.00000,0.270976,...,SummaryType.COLUMN,0,1599,0,0,0,0,NaN,NaN,NaN
density,439.557368,433.943761,445.310933,0,1599,0,0,0,1.00369,0.996747,...,SummaryType.COLUMN,0,1599,0,0,0,0,NaN,NaN,NaN
fixed acidity,96.000023,96.000000,96.004816,0,1599,0,0,0,15.90000,8.319637,...,SummaryType.COLUMN,0,1599,0,0,0,0,NaN,NaN,NaN


## Persisting as a file

After collecting profiles, it is a good practice to store them as a file. This will allow you to later on read them back, merge with future profiles and track how is your data behaving along the way.

In [ ]:
dataset_profile_view.write(path="my_super_awesome_profile.bin")

And that's it, you have just written a profile generated with spark to your local environment! If you wish to upload to different locations, such as s3, whylabs or others, please make sure to check out our [other examples](https://github.com/whylabs/whylogs/tree/mainline/python/examples) page.

Hopefully this tutorial will help you get started to profile and observe your data behaviour in your Spark jobs with almost no friction :)

## Periods in column names

Spark interprets periods in column names as structured fields. This can cause problems when using Spark to profile dataframes created outside of Spark that have periods in their column names. For example:

In [21]:
pandas_df = pd.DataFrame({"col1.mass": [2.3, 4.1, 1.8], "col1.volume": [10.3, 8.7, 6.3]})
spark_df = spark.createDataFrame(pandas_df)
spark_df.show()
collect_column_profile_views(spark_df)

+---------+-----------+
|col1.mass|col1.volume|
+---------+-----------+
|      2.3|       10.3|
|      4.1|        8.7|
|      1.8|        6.3|
+---------+-----------+



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `col1`.`mass` cannot be resolved. Did you mean one of the following? [`col1`.`mass`, `col1`.`volume`].

The error above is caused by Spark misinterpreting the periods in the column names. We can work around this limitation by replacing any periods in the column names:

In [22]:
renamed_df = spark_df.toDF(*(c.replace('.', '_') for c in spark_df.columns))
renamed_df.show()
collect_column_profile_views(renamed_df)

+---------+-----------+
|col1_mass|col1_volume|
+---------+-----------+
|      2.3|       10.3|
|      4.1|        8.7|
|      1.8|        6.3|
+---------+-----------+



{'col1_mass': <whylogs.core.view.column_profile_view.ColumnProfileView at 0x7a9dc6e7c520>,
 'col1_volume': <whylogs.core.view.column_profile_view.ColumnProfileView at 0x7a9dbfdae950>}

⚠ Be sure that any column names specified in whylogs schemas or UDFs match the renamed columns if you use this approach.

## Important note

As you might have seen from the imports, currently this pyspark implementation is the **experimental** phase. We ran some benchmark ourselves with it, and for the sake of example, a `90Gb` dataset with 80M rows could be profiled in under 3 minutes! Cool, right? But we still want more users to try this on their own, see if there are places to be improved and give us feedback before we make it officially **the** spark module here.
Please, feel free to reach out to our [community Slack](https://communityinviter.com/apps/whylabs-community/rsqrd-ai-community) and interact with us there. We will love to hear from you :)